<center><h1>How to bias-adjust time series of climate data using <a href="https://github.com/btschwertfeger/Bias-Adjustment-Cpp" target="_blank">this</a> data structures?</h1></center>

# 0. Preparation
## 0.1 Includes

In [1]:
#include <math.h>
#include <iostream>

In [2]:
#pragma cling add_include_path("/Users/benjamin/repositories/awi-workspace/Bias-Adjustment-Cpp/include/") // change for the specific location of the header files
#pragma cling add_include_path("/Users/benjamin/repositories/awi-workspace/Bias-Adjustment-Cpp/src/")     //     --//--      --//--      --//--      source files
#pragma cling add_library_path("/usr/local/lib")
#pragma cling add_include_path("/usr/local/include")

In [3]:
#include <netcdf>
#pragma cling load("netcdf-cxx4")

In [4]:
#pragma cling load("CMethods.hxx")
#pragma cling load("NcFileHandler.hxx")
#pragma cling load("MathUtils.hxx")

#pragma cling load("CMethods.cxx")
#pragma cling load("NcFileHandler.cxx")
#pragma cling load("MathUtils.cxx")

____
## 0.2 Setup

In [5]:
std::string variable_name = "tas";

In [6]:
NcFileHandler
    ds_obs = NcFileHandler("input_data/observations.nc", variable_name, 3),
    ds_simh = NcFileHandler("input_data/control.nc", variable_name, 3),
    ds_simp = NcFileHandler("input_data/scenario.nc", variable_name, 3); // 3 = number of dimensions (time x lat x lon)

In [7]:
std::vector<float> v_obs_one_loc(ds_obs.n_time);   // observations (control period)
std::vector<float> v_simh_one_loc(ds_obs.n_time);  // simulated (control period)
std::vector<float> v_simp_one_loc(ds_obs.n_time);  // simulated (scenario period)

std::vector<float> v_ls_result(ds_obs.n_time);   // Lienar Scaling result
std::vector<float> v_qdm_result(ds_obs.n_time);  // Quantile Delta Mapping result

____
## 0.3 Select grid box

In [8]:
// select one time series from the data set (i. e. the time series of one location/grid cell)
ds_obs.get_timeseries(v_obs_one_loc, 0, 0);
ds_simh.get_timeseries(v_simh_one_loc, 0, 0);
ds_simp.get_timeseries(v_simp_one_loc, 0, 0);

## 1. Select methods

In [9]:
// select metotds 
CM_Func_ptr_scaling_add apply_adjustment_ls_add = CMethods::get_cmethod_scaling_add("linear_scaling");
CM_Func_ptr_distribution apply_adjustment_qdm = CMethods::get_cmethod_distribution("quantile_delta_mapping");

## 2. Apply methods

In [10]:
// apply additive linear scaling
apply_adjustment_ls_add(
    v_ls_result,                // output vector
    v_obs_one_loc,              // reference data (control period)
    v_simh_one_loc,             // modeled data (control period)
    v_simp_one_loc,             // modeled data (scenario period)
    true                        // interval scaling; i.e. the mean is calculated based on -15 and +15 days for every day of year over all years instead of the mean per long-term month
) 
// note: when not 

In [11]:
// apply quantile delta mapping
apply_adjustment_qdm(
    v_qdm_result,               // output vector
    v_obs_one_loc,              // reference data (control period)
    v_simh_one_loc,             // modeled data (control period)
    v_simp_one_loc,             // modeled data (scenario period)
    "+",                        // kind of adjustment ('+' or '*')
    250                         // number of quantiles to respect
) 

In [12]:
int start = 0, end = 10;

### Observational data (control period 1971 - 2000)

In [13]:
for(unsigned ts = start; ts < end; ts++) std::cout << v_obs_one_loc[ts] << " ";

-24.0942 -24.417 -24.1754 -24.0361 -23.7633 -24.1706 -23.71 -24.5677 -24.654 -23.4294 

In [14]:
std::cout << MathUtils::mean(v_obs_one_loc);

-2.49039

### Model data (control period 1971 - 2000)

In [15]:
for(unsigned ts = start; ts < end; ts++) std::cout << v_simh_one_loc[ts] << " ";

-26.0942 -26.417 -26.1754 -26.0361 -25.7633 -26.1706 -25.71 -26.5677 -26.654 -25.4294 

In [16]:
std::cout << MathUtils::mean(v_simh_one_loc);

-4.49039

### Model data (scenario period 2001 - 2030)

In [17]:
for(unsigned ts = start; ts < end; ts++) std::cout << v_simp_one_loc[ts] << " ";

-25.0942 -25.417 -25.1754 -25.0361 -24.7633 -25.1706 -24.71 -25.5677 -25.654 -24.4294 

In [22]:
std::cout << MathUtils::mean(v_simp_one_loc);

-3.49039

### Linear Scaling-adjusted data (2001 - 2030)

In [23]:
for(unsigned ts = 0; ts < 10; ts++) std::cout << v_ls_result[ts] << " ";

-23.0942 -23.417 -23.1754 -23.0361 -22.7633 -23.1706 -22.71 -23.5677 -23.654 -22.4294 

In [24]:
std::cout << MathUtils::mean(v_ls_result);

// as you can see, the linear scaled result is 2°C warmer than the scenario data set. This
// is why the difference between the observations and the modeled data of the control period
// is adjusted.

-1.49039

### Quantile Delta Mapping-adjusted data (2001 - 2030)

In [20]:
for(unsigned ts = 0; ts < 10; ts++) std::cout << v_qdm_result[ts] << " ";

-23.0842 -23.4073 -23.1656 -23.0243 -22.7518 -23.1609 -22.7013 -23.5562 -23.6325 -22.4175 

In [21]:
std::cout << MathUtils::mean(v_qdm_result);

-1.48977

## 3. Save to file

In [23]:
// Save to new file without any time attributes -> for example when only a part of the time series is adjusted and needs to be saved
/** Note: 
*    The last parameter is n_time, so one can save datasets that does not match the NcFileHandlers` (<ds_simp>) dimensions .
*    All time attributes are missing here, so one have to set them using some other variable.
*/
NcFileHandler ncSaver;
ncSaver.to_netcdf("qdm_result.nc", variable_name, v_qdm_result);

In [24]:
// Saving can also be done by using the ds_simp instance to copy all time attributes and values to the full adjusted time series 
ds_simp.to_netcdf("qdm_result.nc", variable_name, v_qdm_result);

# 4. More usage examples based on Unidata Program Center's NetCDF data strucutres 
References: http://doi.org/10.5065/D6H70CW6

In [25]:
std::cout << ds_simp.time_name << " " << ds_simp.lat_name << " " << ds_simp.lon_name;

time lat lon

In [26]:
std::cout << ds_simp.units << " " << ds_simp.lat_unit << " " << ds_simp.lon_unit;

units degrees_north degrees_east

In [27]:
std::cout << ds_simp.n_time << " " << ds_simp.n_lat << " " << ds_simp.n_lon;

10950 4 2

In [28]:
std::cout << "Latitudes: ";
for(unsigned lat = 0; lat < ds_simp.n_lat; lat++) std::cout << ds_simp.lat_values[lat] << " ";
std::cout << std::endl << "Longitudes: ";
for(unsigned lon = 0; lon < ds_simp.n_lon; lon++) std::cout << ds_simp.lon_values[lon] << " ";
std::cout << std::endl << "First 10 time values: ";
for(unsigned time = 0; time < 10; time++) std::cout << ds_simp.time_values[time] << " ";
std::cout << "...";

Latitudes: 23 24 25 26 
Longitudes: 0 1 
First 10 time values: 0 1 2 3 4 5 6 7 8 9 ...

So why are time values just integers?

-> Because time is counted up by an initial date: 

In [29]:
ds_simp.time_var.getAtts()

{ "calendar" => @0x600000ac40a8, "units" => @0x600000ac5fb8 }

In [30]:
for (std::pair<std::string, netCDF::NcVarAtt> att : ds_simp.time_var.getAtts()) {
    if (att.second.getType().getName() == "char") {
        char value[att.second.getAttLength()];
        att.second.getValues(value);
        std::cout << value;
    }
}

noleapdays since 2001-01-01 00:00:00.000000